<a href="https://colab.research.google.com/github/odoricoveloso/youtube-transcript-scraper/blob/master/captionsScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update && apt upgrade -y && apt autoremove

In [ ]:
#@markdown <center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Selenium_logo.svg/512px-Selenium_logo.svg.png" height="50" alt="Selenium-logo' "/></center>
#@markdown <center><h3>Pre-requisites install</h3></center><br>
!apt install chromium-chromedriver
!pip install selenium
!pip install urllib3==1.25.11

In [ ]:
#@markdown <center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Mount Google Drive</h3></center><br>
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown <br><center><img src="https://img.icons8.com/emoji/192/open-file-folder-emoji.png" height="50" alt="Folder-logo' "/></center>
#@markdown <center><h3>Create project folder</h3></center><br>
import os
os.chdir('/content/drive/MyDrive')
directory = input("Enter folder name: ")
parent_dir = '/content/drive/MyDrive'
path = os.path.join(parent_dir, directory)

# create project folder
if not os.path.exists(path):
  os.mkdir(path)
  print('Directory "%s" created' %directory)
else:
  print('Directory "%s" already exists' %directory)

# create folder "subtitles"
os.chdir('/content/drive/MyDrive/' + directory)
if not os.path.exists('subtitles'):
  os.mkdir('subtitles')
  print('Directory "subtitles" created')
else:
  print('Directory "subtitles" already exists')

In [ ]:
#@markdown <br><center><img src="https://img.icons8.com/officel/160/new-by-copy--v2.png" height="50" alt="File-logo' "/></center>
#@markdown <center><h3>Import csv file to project folder</h3></center><br>
os.chdir('/content/drive/MyDrive/' + directory)
from google.colab import files
uploaded = files.upload()

#<br><center><img src="https://img.icons8.com/officel/160/new-by-copy--v2.png" height="50" alt="File-logo' "/></center>
#<center><h3>JavaScript to keep Colab running</h3></center><br>
Colar o código abaixo no console do navegador (Ctrl + Shift + i) para iniciar a função:
```
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();
```
Colar o código abaixo no console do navegador (Ctrl + Shift + i) para parar a função:
```
stopClickConnect();
```





In [ ]:
#@markdown <br><center><img src="https://img.icons8.com/external-filled-color-icons-papa-vector/156/external-Data-Mining-data-mining-and-big-data.-color.-concept-filled-color-icons-papa-vector.png" height="50" alt="Scrapy-logo' "/></center>
#@markdown <center><h3>Let's Scrapy!</h3></center><br>
# modify these values
filename = next(iter(uploaded)) # filname with video ids
colname = 'videoId'             # column storing video ids
delimiter = ','									# delimiter, e.g. ',' for CSV or '\t' for TAB
waittime = 5										# seconds browser waits before giving up
sleeptime = [2,5]								# random seconds range before loading next video id

#do not modify below
from time import sleep
import csv
import random
import os.path
from datetime import datetime
import pytz
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# logging config
logging.basicConfig(filename = path + '/captions.log', filemode = 'w', level = logging.INFO, format = '%(message)s', force = True)

# datetime config
datetimesp = datetime.now(pytz.timezone('America/Sao_Paulo'))
dt = datetimesp.strftime('%d/%m/%Y %H:%M')

def gettranscript(videoid):

	# check if transcript file already exists	
	writefilename = path + '/subtitles/transcript_' + videoid + '.txt'
	if os.path.isfile(writefilename):
		msg = 'transcript file already exists'
		return msg

	sleep(random.uniform(sleeptime[0],sleeptime[1]))

	options = Options()
	options.add_argument('--headless')
	options.add_argument('--no-sandbox')
	options.add_argument('--disable-dev-shm-usage')

	# Create a new instance of the Firefox driver
	driver = webdriver.Chrome('chromedriver',options=options)

	# navigate to video
	driver.get("https://www.youtube.com/watch?v="+videoid)

	# find options button
	try:
		element = WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="button-shape"]/button/yt-touch-feedback-shape/div/div[2]')))
	except:
		msg = 'could not find options button'
		driver.quit()
		return msg

	# click options button
	try:
		element.click()
	except:
		msg = 'could not click'
		driver.quit()
		return msg

	# find show transcript button
	try:
		element = WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="items"]/ytd-menu-service-item-renderer[2]/tp-yt-paper-item/yt-formatted-string')))
	except:
		msg = 'could not find transcript in options menu'
		driver.quit()
		return msg

	# click show transcript button
	try:
		element.click()
	except:
		msg = 'could not click'
		driver.quit()
		return msg

	# find transctript container
	try:
		element = WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.ID, "segments-container")))
	except:
		msg = 'could not find transcript text'
		driver.quit()
		return msg

	# print(element.text)

	# write transcript in file
	file = open(writefilename,"w")
	file.write(element.text)
	file.close() 

	driver.quit()

	return 'ok'

# read CSV file
csvread = open(filename, newline='\n')
csvreader = csv.DictReader(csvread, delimiter=delimiter, quoting=csv.QUOTE_NONE)
rowcount = len(open(filename).readlines())

for row in csvreader:
	msg = gettranscript(row[colname])
	rowcount -= 1
	logging.info(f'{dt} | {str(rowcount)} | {row[colname]} | {msg}')
	print(dt + ' | ' + str(rowcount) + ' | ' + row[colname] + ' | ' + msg)
 